In [73]:
import pandas as pd
import cplex
from cplex.exceptions import CplexError
from collections import Iterable
import sys
import numpy as np

In [67]:
# Reading the data
#CAC40 = pd.read_excel('./data_if/CAC40.xlsx', index_col = 'Date')
#dowjones = pd.read_excel('./data_if/dowjones.xlsx', index_col = 'Date')
CAC40 = pd.read_excel('./data_if/CAC40.xlsx').drop(columns = ['Date'])
dowjones = pd.read_excel('./data_if/dowjones.xlsx').drop(columns = ['Date'])

2842


In [68]:
# Compute returns
def compute_returns(df):
    returns = pd.DataFrame(data = [])
    l = df.shape[0]
    for asset in df:
        for t in range(1, l):
            returns.at[t - 1, asset] = (df.at[t, asset] - df.at[t - 1, asset]) / df.at[t - 1, asset]
    return returns

In [69]:
def flatten(iterable):
    for el in iterable:
        if isinstance(el, Iterable) and not isinstance(el, str): 
            yield from flatten(el)
        else:
            yield el

In [70]:
def setProblemData(p, index_cor, q):
    
    dim = index_cor.shape[0]
    p.objective.set_sense(p.objective.sense.maximize)
    # define colnames
    X = [["x_{}_{}".format(i, j) for j in range(1, dim + 1)] for i in range(1, dim + 1)]
    Y = ["y_{}".format(i) for i in range(1, dim + 1)]
    
    cor_values = index_cor.values.flatten().tolist()
    # add x_i_j
    my_ub = [1] * dim * dim
    my_lb = [0] * dim * dim
    p.variables.add(obj = cor_values,  names = list(flatten(X)),
                    ub = my_ub, lb = my_lb)
    # add y_i_j
    my_ub = [1] * dim
    my_lb = [0] * dim
    p.variables.add(obj = [0] * dim,  names = Y, ub = my_ub, lb = my_lb)
    # add q constraint
    senses = "E"
    rhs = [q]
    rows = [[Y, [1] * dim]]
    for i in range(dim):
        rows.append([X[i], [1] * dim])
        senses += "E"
        rhs.append(1)
    
    for i in range(dim):
        for j in range(dim):
            rows.append([[X[i][j], Y[j]], [1, -1]])
            senses += "L"
            rhs.append(0)
    #print(rows)
    
    p.linear_constraints.add(lin_expr = rows, senses = senses, rhs = rhs)

In [71]:
def select_assets(index, q):
    try:
        index = compute_returns(index)
        index_cov = index.cov()
        index_cor = index.corr()
        p = cplex.Cplex()
        setProblemData(p, index_cor, q)
        p.solve()
        
        numrows = p.linear_constraints.get_num()
        numcols = p.variables.get_num()
    
        print("solution status : {} : {}".format(p.solution.get_status(), 
                                             p.solution.status[p.solution.get_status()]))
        print("solution value : {}".format(p.solution.get_objective_value()))
        slack = p.solution.get_linear_slacks()
        pi    = p.solution.get_dual_values()
        x     = p.solution.get_values()
        dj = p.solution.get_reduced_costs()
        #print("type x --> :", type(x))
        x = x[-index_cor.shape[0]:]
    
        assets = index.columns
        chosen_assets = []
    
        for i in range(len(x)):
            if x[i] == 1.0:
                #print("asset : ", assets[i])
                chosen_assets.append(assets[i])
        """for i in range(numrows):
            print ("Row %d:  Slack = %10f  Pi = %10f" % (i, slack[i], pi[i]))"""
        """for j in range(numcols):
            print ("Column %d:  Value = %10f Reduced cost = %10f" % (j, x[j], dj[j]))"""
    
        return chosen_assets
    except CplexError as exc:
        print (exc)

In [72]:
my_assets = select_assets(CAC40, 5)

CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.02 sec. (0.43 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =            30.000000
Iteration:    73   Dual objective     =            22.887115
Iteration:   155   Dual objective     =            20.417147
solution status : 1 : optimal
solution value : 20.308610169825496


In [31]:
def matrixToList(Mat):
    MatList = []
    for i in range(Mat.shape[1]):
        MatList.append([range(Mat.shape[1]), (Mat[i]).tolist()])
    return MatList

In [89]:
def geometric_mean(df):
    #gmean = pd.DataFrame(data = [])
    df = df.replace([np.inf, -np.inf], np.nan).dropna()
    L = []
    l = df.shape[0]
    for asset in df:
        p = 1
        T = 0
        for t in range(0, l):
            T += 1
            p *= (1 + df.at[t, asset])
        L.append(pow(p, 1/T) - 1)
        #print(L)
    return L

In [94]:
def markowitz(index, assets_list, R):
    data = index[assets_list]
    data_cov = data.cov()
    data_returns = compute_returns(data)
    gmean_returns = geometric_mean(data_returns)
    print("Asset geometric returns : ",gmean_returns)
    n = len(assets_list)
    
    X = ["x_{}".format(i + 1) for i in range(n)]
    
    qmat = matrixToList(data_cov.values)
    #print(qmat)
    
    try:
    
        p = cplex.Cplex()

        p.set_problem_name("Markowitz Portfolio Optimization")
        p.objective.set_sense(p.objective.sense.minimize)
        p.variables.add( names = X)
        p.objective.set_quadratic(qmat)

        rows = [[X, [1] * n]]
        rows.append([X, gmean_returns])

        p.linear_constraints.add(lin_expr = rows, senses = "EG", rhs = [1, R])

        p.solve()
        
        numrows = p.linear_constraints.get_num()
        numcols = p.variables.get_num()
    
        print("solution status : {} : {}".format(p.solution.get_status(), 
                                             p.solution.status[p.solution.get_status()]))
        print("solution value : {}".format(p.solution.get_objective_value()))
        slack = p.solution.get_linear_slacks()
        pi    = p.solution.get_dual_values()
        x     = p.solution.get_values()
        dj = p.solution.get_reduced_costs()
        print("PORTFOLIO COMPOSITION --> :", x)
    
        """for i in range(numrows):
            print ("Row %d:  Slack = %10f  Pi = %10f" % (i, slack[i], pi[i]))"""
        """for j in range(numcols):
            print ("Column %d:  Value = %10f Reduced cost = %10f" % (j, x[j], dj[j]))"""
    
    except CplexError as exc:
        print (exc)

In [95]:
markowitz(CAC40, my_assets, 0.00035)

Asset geometric returns :  [0.00043680764285980445, 0.00015504168638025284, 0.0002807003844871314, 0.00043412028618616816, 0.0002106388948139859]
CPXPARAM_Read_DataCheck                          1
Number of nonzeros in lower triangle of Q = 10
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.01 sec. (0.00 ticks)
Summary statistics for factor of Q:
  Rows in Factor            = 5
  Integer space required    = 5
  Total non-zeros in factor = 15
  Total FP ops to factor    = 55
Tried aggregator 1 time.
QP Presolve added 0 rows and 5 columns.
Reduced QP has 7 rows, 10 columns, and 30 nonzeros.
Reduced QP objective Q matrix has 5 nonzeros.
Presolve time = 0.05 sec. (0.00 ticks)
Parallel mode: none, using 1 thread for barrier
Number of nonzeros in lower triangle of A*A' = 21
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.01 sec. (0.00 ticks)
Summary statistics for Cholesky factor:
  Rows in Factor            = 7
  Integer space 